## Neo4j Python Driver Examples

### Dependencies

You will need to install the `neo4j` Python package.

```bash
pip install neo4j
```

### Connect

Connect to a Neo4j database using the Neo4j Python driver.

The example connects to the database, verifies the connection, runs a simple Cypher query to count the nodes, and prints the result.

In [ ]:
from neo4j import GraphDatabase

NEO4J_URI = 'neo4j://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'neoneoneo'

# Initialize the Neo4j driver
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(
        NEO4J_USERNAME, 
        NEO4J_PASSWORD
    )
)

# Verify the connection
driver.verify_connectivity()

# Run a simple query to count nodes in the database
records, summary, keys = driver.execute_query(
    "RETURN COUNT {()} AS count"
)

# Get the first record
first = records[0]

# Print the count entry
print(first["count"])   # (3)

# Close the driver
driver.close()

### Execute Cypher Query

Run a Cypher query using the Neo4j Python driver.

Query parameters are used to run the query, and the result is printed.

> You can add the expected data to the database using the _Load CSV data_ example.

In [ ]:
from neo4j import GraphDatabase

NEO4J_URI = 'neo4j://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'neoneoneo'

# Initialize the Neo4j driver
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(
        NEO4J_USERNAME, 
        NEO4J_PASSWORD
    )
)

# Verify the connection
driver.verify_connectivity()

# Define the query
cypher_query = """
MATCH (p:Person)-[:LIVES_IN]->(l:Location)
MATCH (p)-[w:WORKS_AT]->(c:Company)
WHERE l.name = $location
RETURN p.name as name, c.name as company, w.position as position
"""

# Execute the query with a parameter
records, summary, keys = driver.execute_query(
    cypher_query,
    location='London'
)

# When only reading data, you can optimize performance by setting the 
# routing_ parameter to READ mode.

# records, summary, keys = driver.execute_query(
#     cypher_query,
#     routing_='r',
#     location='London'
# )

# Parse the result
for record in records:
    # Print the return values
    print(f"Name: {record['name']}, Company: {record['company']}, Position: {record['position']}")

# Close the driver
driver.close()

### Load CSV Data

Create data from a CSV file.

The example loads data from a CSV file of employees and creates a graph with 3 node labels, `Person`, `Company`, and `Location`, connected by `WORKS_AT` and `LIVES_IN` relationships.

CSV file - [employees.csv](../data/employees.csv):

```
id,name,company,position,location,gov_id
1,"John Doe","TechCorp","Engineer","Delhi",123456789
2,"Jane Smith","TechCorp","Manager","Sydney",987654321
```

Graph:

```
+-----------+         WORKS_AT         +-----------+
|  Person   |------------------------->|  Company  |
+-----------+                          +-----------+
      |
      |
      | LIVES_IN
      v
+-----------+
| Location  |
+-----------+
```

In [ ]:
import os
import csv
from neo4j import GraphDatabase

FILE_PATH = os.path.join('..','data','employees.csv')

NEO4J_URI = 'neo4j://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'neoneoneo'

# Initialize the Neo4j driver
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(
        NEO4J_USERNAME, 
        NEO4J_PASSWORD
    )
)

# Verify the connection
driver.verify_connectivity()

# Cypher query to create the data
cypher_query = """
MERGE (p:Person {id: toInteger($id), name: $name, governmentId: $gov_id})
MERGE (l:Location {name: $location})
MERGE (c:Company {name: $company})
MERGE (p)-[:LIVES_IN]->(l)
MERGE (p)-[:WORKS_AT {position: $position}]->(c)
"""

# Load the CSV file
with open(FILE_PATH, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        
        # Execute the query with parameters
        records, summary, keys = driver.execute_query(
            cypher_query,
            id=row['id'],
            name=row['name'],
            gov_id=row['gov_id'],
            location=row['location'],
            company=row['company'],
            position=row['position']
        )

        # Alternatively, you can pass the row as parameters
        # records, summary, keys = driver.execute_query(
        #     cypher_query,
        #     parameters_= row
        # )

        print(summary.counters)

driver.close()

### Export to Pandas DataFrame

Run a Cypher query and export the results to a Pandas DataFrame.

In [ ]:
from neo4j import GraphDatabase, Result

NEO4J_URI = 'neo4j://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'neoneoneo'

# Initialize the Neo4j driver
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(
        NEO4J_USERNAME, 
        NEO4J_PASSWORD
    )
)

# Verify the connection
driver.verify_connectivity()

# Define the query
cypher_query = """
MATCH (p:Person)-[:LIVES_IN]->(l:Location)
MATCH (p)-[w:WORKS_AT]->(c:Company)
RETURN 
    p.name as name, 
    c.name as company, 
    w.position as position, 
    l.name as location
"""

# Execute the query using the Result transformer 
df = driver.execute_query(
    cypher_query,
    result_transformer_=Result.to_df,
    location='London'
)

# Print the DataFrame
print(df)

# Close the driver
driver.close()

### Transactions

Execute a function in a single transaction.

In [ ]:
from neo4j import GraphDatabase

NEO4J_URI = 'neo4j://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'neoneoneo'

# Initialize the Neo4j driver
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(
        NEO4J_USERNAME, 
        NEO4J_PASSWORD
    )
))

# Verify the connection
driver.verify_connectivity()

# Create a session to run a transaction
with driver.session() as session:

    # Create a work unit for the transaction
    def create_person(tx, name, age):
        result = tx.run("""
        CREATE (p:Person {name: $name, age: $age})
        RETURN p
        """, name=name, age=age)

        return result.consume()

    # Execute transaction function
    summary = session.execute_write(create_person, name='Alice', age=30)
    print(summary.counters.nodes_created, 'node(s) created.')


# Close the driver
driver.close()